# アカウントのテキストと画像をもらう

In [ ]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token)
        self.count = 5


    @property
    def text_zget(self):
        """_summary_
        usernameにアカウント名を入力

        画像を取る場合は以下を有効にする
        for tweets_img in target_tweets.includes['media']:

        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []

        target_response = self.client.get_user(username='hikarinspecial') # usernameにアカウント名を入力
        target_tweets = self.client.get_users_tweets(target_response.data['id'], expansions=["attachments.media_keys"], max_results=10, media_fields=['preview_image_url', 'url'], exclude='retweets')

        while True:
            print(target_tweets)
            next_token = target_tweets.meta['next_token']

            # テキストを取る
            for tweets in target_tweets.data:
                text = tweets.data['text']
                print(text)

            # 画像を取る
            for tweets_img in target_tweets.includes['media']:
                img_url = tweets_img['url']
                print(img_url)

            tweet['tweet'].append({"text": text,"img": img_url})
            print(tweet)

            with open('imada_mio.json', 'w', encoding='utf-8') as f:
                json.dump(tweet, f, indent=4, ensure_ascii=False)

            target_tweets = self.client.get_users_tweets(target_response.data['id'], expansions=["attachments.media_keys"], max_results=10, media_fields=['preview_image_url', 'url'], exclude='retweets', pagination_token=next_token)

            if next_token is None:
                break



t = Tweet_text()
t.text_get


        # for tweets in tweepy.Paginator(self.client.get_users_tweets, target_response.data['id'], max_results=10, media_fields=['preview_image_url', 'url'], expansions=['attachments.media_keys'], tweet_fields=["entities"]).flatten(limit=250):


# v1でテキストと画像取得できた

- v2 テキストと画像取得しようとしたけどうまくいかない→ v1.1でやった方が楽という結論
- v1でOK

In [71]:
import tweepy
import API_config_katudon
import json
import requests
import re
import os


class Tweet_text:

    def __init__(self, username):

        auth = tweepy.OAuthHandler(API_config_katudon.API_KEY, API_config_katudon.API_SECRET)
        auth.set_access_token(API_config_katudon.ACCESS_TOKEN, API_config_katudon.ACCESS_TOKEN_SECRET)
        self.api = tweepy.API(auth, wait_on_rate_limit=True)
        self.__username = username

    def text_image_get(self):


        count_no = 200
        tweet: dict[dict] = {}
        tweet['tweet'] = []

        results: iter = tweepy.Cursor(self.api.user_timeline, # タイムラインの取得
                    id=self.__username, # 取得対象のユーザーを指定
                    include_entities=True, # 省略されたリンクを全て取得
                    tweet_mode='extended', # 省略されたツイートを全て取得
                    lang='ja',
                    exclude_replies=False,
                    include_rts=False).items(limit=count_no) # 取得件数を指定

        index = 1

        for result in results:
            if 'media' in result.entities:
                image_url = result.extended_entities['media'][0]['media_url']
                texts = result.full_text
                text = texts.split()
                tweet_text = text[0]

                print(tweet)

                img_response = requests.get(image_url)
                match img_response.status_code:
                    case 200:
                        os.makedirs(f'{self.__username}', mode=0o777, exist_ok=True)
                        with open(f'{self.__username}/{index}.jpg', 'wb') as image:
                            image.write(img_response.content)

                        tweet['tweet'].append({"text": tweet_text, 'img': image_url, 'img_file': f'{index}.jpg'})
                        index += 1

                with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
                    json.dump(tweet, f, indent=4, ensure_ascii=False)

In [ ]:
t = Tweet_text('erika_1837')
t.text_image_get()

In [ ]:
load = json.load(open('erika_1837.json', 'r'))
l = load['tweet']


for i in l:
    print(i['img'])
    img_response = requests.get(i['img'])

NG エロアカウントからテキストだけもらう

In [1]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self, username):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token, wait_on_rate_limit=True)
        self.count = 5
        self.__username = username

    def text_get(self):
        """_summary_
        usernameにアカウント名を入力
        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []

        target_response = self.client.get_user(username=self.__username) # usernameにアカウント名を入力
        # テキストだけ
        target_tweets = self.client.get_users_tweets(target_response.data['id'], max_results=100, exclude=['retweets', 'replies'])
        print(target_tweets)

        while True:
            next_token = target_tweets.meta['next_token']

            for tweets in target_tweets.data:
                text = tweets.data['text']
                tweet['tweet'].append({"text": text})

            with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
                json.dump(tweet, f, indent=4, ensure_ascii=False)

            target_tweets = self.client.get_users_tweets(target_response.data['id'], max_results=100, exclude=['retweets', 'replies'])

            if next_token is None:
                break

        # if next_token:
        #     while True:
        #         target_tweets_next = self.client.get_users_tweets(target_response.data['id'], max_results=100, exclude=['retweets', 'replies'], pagination_token=next_token)
        #         next_token = target_tweets_next.meta['next_token']

        #         for tweets in target_tweets_next.data:
        #             text = tweets.data['text']
        #             print(text)

        #             tweet['tweet'].append({"text": text})
        #             print(tweet)

        #             with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
        #                 json.dump(tweet, f, indent=4, ensure_ascii=False)

        #         if next_token is None:
        #             break

        # for tweets in tweepy.Paginator(self.client.get_users_tweets, target_response.data['id'], max_results=10, media_fields=['preview_image_url', 'url'], expansions=['attachments.media_keys'], tweet_fields=["entities"]).flatten(limit=250):


In [ ]:
t = Tweet_text('hikarinspecial')
t.text_get

# テキストだけ取得（paginatorを使う）

In [15]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self, username):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token, wait_on_rate_limit=True)
        self.count = 5
        self.__username = username

    def text_get(self):
        """_summary_
        usernameにアカウント名を入力
        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []
        target_response = self.client.get_user(username=self.__username) # usernameにアカウント名を入力

        for tweets in tweepy.Paginator(self.client.get_users_tweets,target_response.data['id'], exclude=['retweets', 'replies'],max_results=100).flatten(limit=1000):

            text = tweets.data['text']
            tweet['tweet'].append({"text": text})

            with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
                json.dump(tweet, f, indent=4, ensure_ascii=False)


In [18]:
t = Tweet_text('Lillian46612363')
t.text_get()

In [2]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self, username):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token, wait_on_rate_limit=True)
        self.__username = username

    def text_get(self):
        """_summary_
        usernameにアカウント名を入力
        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []
        target_response = self.client.get_user(username=self.__username) # usernameにアカウント名を入力

        for tweets in tweepy.Paginator(self.client.get_users_tweets,target_response.data['id'], exclude=['replies'],max_results=100).flatten(limit=3000):

            self.client.delete_tweet(tweets.id)

            # JSON保存
            # text = tweets.data['text']
            # tweet['tweet'].append({"text": text})
            # with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
            #     json.dump(tweet, f, indent=4, ensure_ascii=False)

t = Tweet_text('katudon')
t.text_get()

# 裏垢テキストと画像

In [ ]:
import tweepy
import API_config_katudon
import json


class Tweet_text:
    """v2"""

    def __init__(self):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token)
        self.count = 5


    def text_media_get(self, username):
        """_summary_
        usernameにアカウント名を入力

        画像を取る場合は以下を有効にする
        for tweets_img in target_tweets.includes['media']:

        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []

        target_response = self.client.get_user(username=username)
        target_tweets = self.client.get_users_tweets(target_response.data['id'], expansions=["attachments.media_keys"], max_results=10, media_fields=['preview_image_url', 'url'], exclude=['retweets', 'replies'])
        next_token = target_tweets.meta['next_token']

        while True:
            print(target_tweets)
            try:
                if len(target_tweets.includes) == 0:
                    for tweets in target_tweets.data:
                        text = tweets.data['text']
                        tweet['tweet'].append({"text": text})
                        print(text)
                elif target_tweets.includes:
                    for tweets_img in target_tweets.includes['media']:
                        img_url = tweets_img['url']
                        print(img_url)
                        tweet['tweet'].append({"img": img_url})
                        print(tweet)

                target_tweets = self.client.get_users_tweets(target_response.data['id'], expansions=["attachments.media_keys"], max_results=10, media_fields=['preview_image_url', 'url'], exclude=['retweets', 'replies'], pagination_token=next_token)
                next_token = target_tweets.meta['next_token']

            except KeyError as e:
                print(f'[text_media_get] Exception: {e}')
                break

        return tweet


username = 'poyopoyoman88'
t = Tweet_text()
tweet = t.text_media_get(username)

with open(f'/home/don/py/DMM/twitter_pic_download/tweet_{username}.json', 'w', encoding='utf-8') as f:
            json.dump(tweet, f, indent=4, ensure_ascii=False)



In [4]:
import tweepy
import API_config_katudon
import json
import requests
import re
import os


class Tweet_text:
    """v1"""

    def __init__(self, username):

        auth = tweepy.OAuthHandler(API_config_katudon.API_KEY, API_config_katudon.API_SECRET)
        auth.set_access_token(API_config_katudon.ACCESS_TOKEN, API_config_katudon.ACCESS_TOKEN_SECRET)
        self.api = tweepy.API(auth, wait_on_rate_limit=True)
        self.__username = username

    def text_image_get_tojson(self):


        count_no = 5000
        tweet: dict[dict] = {}
        tweet['tweet'] = []

        results: iter = tweepy.Cursor(self.api.user_timeline, # タイムラインの取得
                    id=self.__username, # 取得対象のユーザーを指定
                    include_entities=True, # 省略されたリンクを全て取得
                    tweet_mode='extended', # 省略されたツイートを全て取得
                    lang='ja',
                    exclude_replies=False,
                    include_rts=False).items(limit=count_no) # 取得件数を指定

        index = 1

        for result in results:
            if 'media' in result.entities:
                image_url = result.extended_entities['media'][0]['media_url']
                texts = result.full_text
                text = texts.split()
                tweet_text = text[0]

                print(tweet)

                img_response = requests.get(image_url)
                match img_response.status_code:
                    case 200:
                        os.makedirs(f'/home/don/py/DMM/twitter_pic_download/{self.__username}', mode=0o777, exist_ok=True)
                        with open(f'/home/don/py/DMM/twitter_pic_download/{self.__username}/{index}.jpg', 'wb') as image:
                            image.write(img_response.content)

                        tweet['tweet'].append({"text": tweet_text, 'img': image_url, 'img_file': f'{index}.jpg'})
                        index += 1

                # with open(f'/home/don/py/DMM/twitter_pic_download/{self.__username}.json', 'w', encoding='utf-8') as f:
                #     json.dump(tweet, f, indent=4, ensure_ascii=False)

            elif 'media' not in result.entities:
                texts = result.full_text
                text = texts.split()
                tweet_text = text[0]
                print(tweet_text)

                tweet['tweet'].append({"text": tweet_text})
                index += 1

        with open(f'/home/don/py/DMM/twitter_pic_download/{self.__username}.json', 'w', encoding='utf-8') as f:
            json.dump(tweet, f, indent=4, ensure_ascii=False)

In [5]:
t = Tweet_text('poyopoyoman88')
t.text_image_get_tojson()

Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang
Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang


ホテルもいいけどお家でするのもあり。私の家来ますか？💕
今日は40代の男性とデートしてきましたよ♪体の相性よかったからまた次に会う約束もしちゃった😌💗笑
反響大きすぎて嬉しい🥰💕
ホテル代なら私が出しますし、お仕事や学校忙しくて遠くに来れないなら私が会いに行きます。
オフぱこしたいお兄さんいますか？
ホテル代なら私が出しますし、お仕事や学校忙しくて遠くに来れないなら私が会いに行きます。
ホテル代は私が出すからホテルついてきてくれるような人いないかなぁ〜
ホテル代とかデート代は私が出すし、住みが遠かったら新幹線で会いに行くので私とホテルデートしたい人はラブリツして？🥺
@CHkpsNrDwtXiMho
私ひとり暮らししてるんですけど誰か家来ませんか？
オフぱこしたいお兄さんいますか？
むらむらしちゃうなぁ🥺💓
お店の女の子と会うよりも私とせックスしよ？ホテル代も払ってあげるし、それなりに経験やテクニックもあるので会って後悔はさせないよ😌✨
おふパコデートしたい人いますか？
シンプルにせフレ募集です。


Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang


騎乗位の練習したいんですけど相手してくれる人いませんか？ちなみに激しめに動くから覚悟してね？笑
おじさんといちゃいちゃしながらえッチしたい…💓
{'tweet': [{'text': 'ホテルもいいけどお家でするのもあり。私の家来ますか？💕'}, {'text': '今日は40代の男性とデートしてきましたよ♪体の相性よかったからまた次に会う約束もしちゃった😌💗笑'}, {'text': '反響大きすぎて嬉しい🥰💕'}, {'text': 'ホテル代なら私が出しますし、お仕事や学校忙しくて遠くに来れないなら私が会いに行きます。'}, {'text': 'オフぱこしたいお兄さんいますか？'}, {'text': 'ホテル代なら私が出しますし、お仕事や学校忙しくて遠くに来れないなら私が会いに行きます。'}, {'text': 'ホテル代は私が出すからホテルついてきてくれるような人いないかなぁ〜'}, {'text': 'ホテル代とかデート代は私が出すし、住みが遠かったら新幹線で会いに行くので私とホテルデートしたい人はラブリツして？🥺'}, {'text': '@CHkpsNrDwtXiMho'}, {'text': '私ひとり暮らししてるんですけど誰か家来ませんか？'}, {'text': 'オフぱこしたいお兄さんいますか？'}, {'text': 'むらむらしちゃうなぁ🥺💓'}, {'text': 'お店の女の子と会うよりも私とせックスしよ？ホテル代も払ってあげるし、それなりに経験やテクニックもあるので会って後悔はさせないよ😌✨'}, {'text': 'おふパコデートしたい人いますか？'}, {'text': 'シンプルにせフレ募集です。'}, {'text': '騎乗位の練習したいんですけど相手してくれる人いませんか？ちなみに激しめに動くから覚悟してね？笑'}, {'text': 'おじさんといちゃいちゃしながらえッチしたい…💓'}]}
せフレになってくれる人いませんか？
会いたい人はこっちにDMを送ってください🥺💓
おじさんとえッチしたい気分…
フォロワーさんと会いたいんですけど、立候補してくれる人はいませんか？
SNSでの出会いに抵抗ないよって人はぜひ会ってください☺️💓
私とお泊まりでえッチなことしたい人はいませんか？
私とホテルデートしたい人いますか？
初め

Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang


...

@soccerboy851
@TIO6APxUuaRqmBi
@TIO6APxUuaRqmBi
@yuuuu050331
@3lBEvX8ejRtojYs
@D44Uac
@G1970RX782XYZ
@3lBEvX8ejRtojYs
@D44Uac
@3lBEvX8ejRtojYs
@LDH16097085
@LDH16097085
@LDH16097085
@m_otoko072
@LDH16097085
@m_otoko072
@LDH16097085
@Y1msGutSMvbVtOt
@3lBEvX8ejRtojYs
@m_otoko072


Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang


@yLhkxLy4HVQGH0f
@LDH16097085
@LDH16097085
@yasaburooooo
@soccerboy851
@yuuuu050331
@BStj9L1vaAHh5dw
@soccerboy851
@BStj9L1vaAHh5dw
@soccerboy851
@BStj9L1vaAHh5dw
@soccerboy851
@BStj9L1vaAHh5dw
@Y1msGutSMvbVtOt
@soccerboy851
@yuuuu050331
@ZRnzKnIVKTueEi5
@Y1msGutSMvbVtOt
@Y1msGutSMvbVtOt
@Y1msGutSMvbVtOt


Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang


@Y1msGutSMvbVtOt
@Y1msGutSMvbVtOt
@BStj9L1vaAHh5dw
@XoiJAjQfS1PLYfc
@XoiJAjQfS1PLYfc
@XoiJAjQfS1PLYfc
@Natu_ogu
@wagmbPwegpwjpt
@XQkoqOwG8Zuf32e
@Y1msGutSMvbVtOt
@XQkoqOwG8Zuf32e
@XQkoqOwG8Zuf32e
@D44Uac
@XQkoqOwG8Zuf32e
